# SRGAN

Implementation of Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network.

Paper: https://arxiv.org/abs/1609.04802  
Github Reference: https://github.com/eriklindernoren/Keras-GAN/blob/master/srgan/srgan.py

In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import sys
import numpy as np
import os
import math
import time
import keras.backend as K

from pixelshuffle import PixelShuffler
from keras.utils import conv_utils
from keras.engine.topology import Layer

import tensorflow as tf
import tempfile
import h5py
from scipy.misc import imresize
from keras.preprocessing import image as image_utils

import telegram_send

Using TensorFlow backend.


### Load the data

In [2]:
# import the necessary packages

# Data of form (N,H,W,C)

"""
Load HR/LR images of form (N,H,W,C), where N = index (eg; image1, ...), H = height, W = width and C = channels.

path = glob("./path_of_dataset_location")

"""
def loaddata(path):
    #Load HDF5 dataset
    with h5py.File(path, 'r') as f:
    # f = h5py.File('imagenet10.hdf5', 'r')
        x = [i[...] for i in f["x"].values()]
        y = f["y"][...]
        class_names = f["metadata"][...]
    print("Class names:",class_names)
    print("X data size:", len(x))

    ###PINARDY: Smaller dataset sample for testing
    # idx_range = 9557
    x = np.asarray(x)

    y_oh = np.eye(len(class_names))[y]
    print ("Y one hot shape:",y_oh.shape)
    return (x, y, y_oh)

x_train, y_train, y_train_oh = loaddata('dataset/imagenet10_train.hdf5')
x_test, y_test, y_test_oh = loaddata('dataset/imagenet10_test.hdf5')
x_valid, y_valid, y_valid_oh = loaddata('dataset/imagenet10_valid.hdf5')

Class names: [b'bicycle' b'bird' b'bus' b'car' b'cat' b'cow' b'dog' b'horse' b'person'
 b'sheep']
X data size: 8757
Y one hot shape: (8757, 10)
Class names: [b'bicycle' b'bird' b'bus' b'car' b'cat' b'cow' b'dog' b'horse' b'person'
 b'sheep']
X data size: 400
Y one hot shape: (400, 10)
Class names: [b'bicycle' b'bird' b'bus' b'car' b'cat' b'cow' b'dog' b'horse' b'person'
 b'sheep']
X data size: 400
Y one hot shape: (400, 10)


In [3]:
def train_print(imgs_lr,fake_hr,imgs_hr):
    imgs_lr = (imgs_lr - imgs_lr.min())/(imgs_lr.max() - imgs_lr.min())
    fake_hr = (fake_hr - fake_hr.min())/(fake_hr.max() - fake_hr.min())
    imgs_hr = (imgs_hr - imgs_hr.min())/(imgs_hr.max() - imgs_hr.min())
    plt.close()
    plt.subplot(1,3,1)
    plt.imshow(imgs_lr)
    plt.subplot(1,3,2)
    plt.imshow(fake_hr)
    plt.subplot(1,3,3)
    plt.imshow(imgs_hr)
    plt.show()
    
def save_imgset(imgs_lr,fake_hr,imgs_hr,path,epoch, only_fake = True):
    if not only_fake:
        imgs_lr = (imgs_lr - imgs_lr.min(axis=(1,2,3),keepdims=True))/(imgs_lr.max(axis=(1,2,3),keepdims=True) - imgs_lr.min(axis=(1,2,3),keepdims=True))
        fake_hr = (fake_hr - fake_hr.min(axis=(1,2,3),keepdims=True))/(fake_hr.max(axis=(1,2,3),keepdims=True) - fake_hr.min(axis=(1,2,3),keepdims=True))
        imgs_hr = (imgs_hr - imgs_hr.min(axis=(1,2,3),keepdims=True))/(imgs_hr.max(axis=(1,2,3),keepdims=True) - imgs_hr.min(axis=(1,2,3),keepdims=True))
        length = imgs_lr.shape[0]
        fig = plt.figure(figsize=(3,length),dpi=300)
        fig.suptitle("Epoch: %d"%epoch, fontsize=24)
        fig.subplots_adjust(wspace=0.1, hspace=0.1)
        for i in range(length):
            ax = fig.add_subplot(length, 3, i*3 + 1)
            ax.axis("off")
            ax.imshow(imgs_lr[i])
            ax = fig.add_subplot(length, 3, i*3 + 2)
            ax.axis("off")
            ax.imshow(fake_hr[i])
            ax = fig.add_subplot(length, 3, i*3 + 3)
            ax.axis("off")
            ax.imshow(imgs_hr[i])
        plt.savefig(path)
        plt.close()
    else:
        fake_hr = (fake_hr - fake_hr.min(axis=(1,2,3),keepdims=True))/(fake_hr.max(axis=(1,2,3),keepdims=True) - fake_hr.min(axis=(1,2,3),keepdims=True))
        length = fake_hr.shape[0]
        fig = plt.figure(figsize=(3,length),dpi=300)
        fig.suptitle("Epoch: %d"%epoch, fontsize=24)
        fig.subplots_adjust(wspace=0.1, hspace=0.1)
        for i in range(length):
            ax = fig.add_subplot(length, 3, i*3 + 2)
            ax.axis("off")
            ax.imshow(fake_hr[i])
        plt.savefig(path)
        plt.close()

In [4]:
#Params
usevgg = True
Ghidden = [64,256]
Dhidden = [64, 128, 256, 512, 1024]
N_CHANNELS = 3
downsample = 4
PIXSHUF_ITER = int(np.log2(downsample))
print("Downsample:", downsample, "PixShuff:", PIXSHUF_ITER, "Use VGG Loss:",usevgg)
EPOCH=100
mb_size = 16
PERIOD = x_train.shape[0]//mb_size
SAVE_INTERVAL = 10

Downsample: 4 PixShuff: 2 Use VGG Loss: True


In [5]:
#SRGAN / Classifier
def img_crop_downsample(img, downscale):  
    ht, wt = 224,224
    in_ht, in_wt = img.shape[0:2]
    if in_ht < in_wt:
        img = imresize(img, (ht,int(in_wt*ht/in_ht)))
        in_ht, in_wt = img.shape[0:2]
    else:
        img = imresize(img, (int(in_ht*wt/in_wt),wt))
        in_ht, in_wt = img.shape[0:2]
#     if in_ht < ht:
#         img = imresize(img, (ht,int(wt*ht/in_ht)))
#         in_ht, in_wt = img.shape[0:2]
#     if in_wt < wt:
#         img = imresize(img, (int(ht*wt/in_wt),wt))
#         in_ht, in_wt = img.shape[0:2]
        
    dx_ht = in_ht - ht
    dx_wt = in_wt - wt
#     print(img.shape)
    low_ht = np.random.randint(0, dx_ht + 1)
    low_wt = np.random.randint(0, dx_wt + 1)
    
    hr = img[low_ht:low_ht+ht, low_wt:low_wt+ht, :]
    lr = imresize(hr, 1/downscale)
    
    #Normalize v1
#     hr_mean = hr.mean(axis=(0,1), keepdims=True)
#     hr_std = hr.std(axis=(0,1), keepdims=True)
#     hr = (hr-hr_mean)/hr_std
    
#     lr_mean = lr.mean(axis=(0,1), keepdims=True)
#     lr_std = lr.std(axis=(0,1), keepdims=True)
#     lr = (lr-lr_mean)/lr_std

    #Normalize v2 (HR[-1,1], LR[0,1])
    hr = ((hr - hr.min())/(hr.max() - hr.min()) * 2) - 1
    lr = (lr - lr.min())/(lr.max() - lr.min())
    return hr,lr

In [6]:
#RUN ONCE ONLY
save_imgs_hr, save_imgs_lr = list(zip(*map(img_crop_downsample, x_train[16:64], np.ones(16)*downsample)))
save_imgs_hr = np.asarray(save_imgs_hr)
save_imgs_lr = np.asarray(save_imgs_lr)

for f in range(2):
    save_imgset(None,save_imgs_hr[f*4:(f+1)*4],None,'./trained_images/%d_hr'%(f),-1)
    save_imgset(None,save_imgs_lr[f*4:(f+1)*4],None,'./trained_images/%d_lr'%(f),-1)
    

/home/student03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if __name__ == '__main__':
/home/student03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/student03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [7]:
# RGB_MEAN_PIXELS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3)).astype(np.float32)
RGB_MEAN_PIXELS = 127.5
DEFAULT_IMAGE_SHAPE = (None,224,224,3)

class VGG19():
    """
    A class that builds a TF graph with a pre-trained VGG19 model (on imagenet)
    Also takes care of preprocessing. Input should be a regular RGB image (0-255)
    """
    def __init__(self, image_shape=DEFAULT_IMAGE_SHAPE, input_tensor=None, name = '_'):
        self.name = name
        self.image_shape = image_shape
        self._build_graph(input_tensor)

    def _build_graph(self, input_tensor):
        with tf.Session() as sess:
            with tf.variable_scope('VGG19' + self.name):
                with tf.name_scope('inputs'):
                    if input_tensor is None:
                        input_tensor = tf.placeholder(tf.float32, shape=self.image_shape, name='input_img')
                    else:
                        assert self.image_shape[1:] == input_tensor.shape[1:]
                    self.input_tensor = input_tensor

                with tf.name_scope('preprocessing'):
#                     img = self.input_tensor - RGB_MEAN_PIXELS
                    img = self.input_tensor * RGB_MEAN_PIXELS
                    img = tf.reverse(img, axis=[-1])

                with tf.variable_scope('model'):
                    self.vgg19 = tf.contrib.keras.applications.VGG19(weights='imagenet',
                        include_top=False, input_tensor=img)

                self.outputs = { l.name: l.output for l in self.vgg19.layers }

            self.vgg_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='VGG19%s/model'%self.name)

            with tempfile.NamedTemporaryFile() as f:
                self.tf_checkpoint_path = tf.train.Saver(self.vgg_weights).save(sess, f.name)

        self.model_weights_tensors = set(self.vgg_weights)


    def load_weights(self,sess):
#         sess = tf.get_default_session()
        tf.train.Saver(self.vgg_weights).restore(sess, self.tf_checkpoint_path)

    def __getitem__(self, key):
        return self.outputs[key]


In [8]:
#Layers
def conv2d(inp, n_filter, name, filtersize = (3,3),strides=[1, 1, 1, 1], bias = True):
    with tf.variable_scope(name):
        W = tf.Variable(tf.truncated_normal([filtersize[0], filtersize[1], inp.shape.as_list()[-1], n_filter], stddev=0.01))
        if bias:
            b = tf.Variable(tf.constant(0.1, shape=[n_filter]))
            conv = tf.nn.bias_add(tf.nn.conv2d(inp, W, strides, padding='SAME'), b)
        else:
            conv = tf.nn.conv2d(inp, W, [1,strides[0],strides[1],1], padding='SAME')
        return conv

def residualblk(inp, activation, name):
    n_filter = inp.shape.as_list()[-1]
    conv1 = activation(bn(conv2d(inp, n_filter, name+'_conv1')))
    conv2 = conv2d(conv1, n_filter, name+'_conv2')
    return tf.add(conv2, inp)

def fc(inp, n_filter, name, bias = True):
    with tf.variable_scope(name):
        shape = inp.get_shape().as_list()
        dim = 1
        for d in shape[1:]:
            dim *= d
        x = tf.reshape(inp, [-1, dim])
        W = tf.Variable(tf.truncated_normal([dim, n_filter], stddev=0.01))
        if bias:
            b = tf.Variable(tf.constant(0.1, shape=[n_filter]))
            fc = tf.nn.bias_add(tf.matmul(x, W), b)
        else:
            fc =tf.matmul(x, W)
        return fc

def PRelu(u): #Currently relu, consider parametric relu.
    return tf.keras.layers.PReLU()(u)
            
def LRelu(u, a = 0.2):
    return tf.maximum(a * u, u)

def bn(u):
    mean, variance = tf.nn.moments(u, axes=[0, 1, 2])
    return tf.nn.batch_normalization(u, mean, variance, None, None, 1e-5)

def _phase_shift(I, r):
    bsize, a, b, c = I.get_shape().as_list()
    bsize = tf.shape(I)[0] # Handling Dimension(None) type for undefined batch dim
    X = tf.reshape(I, (bsize, a, b, r, r))
    X = tf.transpose(X, (0, 1, 2, 4, 3))
    X = tf.split(X, a, 1)
    X = tf.concat([tf.squeeze(x, axis=1) for x in X], 2)
    X = tf.split(X, b, 1)
    X = tf.concat([tf.squeeze(x, axis=1) for x in X], 2)
    return tf.reshape(X, (bsize, a*r, b*r, 1))

def SubpixelConv2d(X, r, n_out_channel):
    _err_log = ''
    if n_out_channel > 1:
        assert int(X.get_shape()[-1]) == (r ** 2) * n_out_channel, _err_log
        Xc = tf.split(X, n_out_channel, 3)
        X = tf.concat([_phase_shift(x, r) for x in Xc], 3)
    elif n_out_channel == 1:
        assert int(X.get_shape()[-1]) == (r ** 2), _err_log
        X = _phase_shift(X, r)
    else:
        print("Error channel size")
    if _err_log != None:
        print(_err_log)
    return X

#Model input placeholder
X = tf.placeholder(tf.float32, shape=(None, 224, 224, 3), name = 'X')
Z = tf.placeholder(tf.float32, shape=(None, 224//downsample, 224//downsample, 3), name = 'Z')

#Generator
B_ITER = 16 #Based on SRGAN paper
G0 = PRelu(conv2d(Z, Ghidden[0], 'G0', bias=False))
G_B = residualblk(G0, PRelu, 'G_B0') #First RB
for ind in range(1,B_ITER): #Rest of RB
    G_B = residualblk(G_B, PRelu, 'G_B%d'%ind)

Gsum = tf.add(bn(conv2d(G_B, Ghidden[0], 'G1')), G0)
for ind in range(PIXSHUF_ITER): #iterate number of pix shuf based on SR scale
    G2 = conv2d(Gsum, Ghidden[1], 'G%d'%(ind+2), bias=False)
    Gsum = PRelu(SubpixelConv2d(G2, 2, Ghidden[1] // 4))

G = tf.nn.tanh(conv2d(Gsum, N_CHANNELS, 'G%d'%(ind+3), filtersize=(9,9), bias=False), name = "generator")

#Discriminator
def discriminator(xx):
    for ind in range(4):
        if ind == 0: #Exceptional case for first single stride conv2d. No bn, no bias
            Ds1 = LRelu(conv2d(xx, Dhidden[ind], 'D%d_1'%ind, strides=[1, 1, 1, 1], bias=False))
        else:
            Ds1 = LRelu(bn(conv2d(xx, Dhidden[ind], 'D%d_1'%ind, strides=[1, 1, 1, 1])))
        xx = LRelu(bn(conv2d(Ds1, Dhidden[ind], 'D%d_2'%ind, strides=[1, 2, 2, 1])))

    D_fc0 = LRelu(fc(xx, Dhidden[4], 'D_fc0'))
    D_fc1 = fc(D_fc0, 1, 'D_fc1')
    return tf.nn.sigmoid(D_fc1), D_fc1

D_fake, D_logit_fake = discriminator(G)
D_real, D_logit_real = discriminator(X)
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
Dloss = D_loss_real + D_loss_fake

Genloss = 1e-3 * tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
mse = tf.reduce_mean(tf.reduce_mean(tf.squared_difference(G, X), [1, 2, 3]))

if usevgg:
    vgg_hr_model = VGG19(input_tensor = X, name = '_X')
    vgg_hr = tf.identity(vgg_hr_model['block2_conv2'], name='my_output')
    vgg_gen_model = VGG19(input_tensor = G, name = '_G')
    vgg_gen = tf.identity(vgg_gen_model['block2_conv2'], name='my_output')
    vggloss = 5e-6 * tf.reduce_mean(tf.reduce_mean(tf.squared_difference(vgg_gen, vgg_hr), [1, 2, 3]))
else:
    vggloss = tf.constant(0.0)

Gloss = Genloss + mse + vggloss

vars = tf.trainable_variables()
Dvars = [v for v in vars if v.name.startswith("D")]
Gvars = [v for v in vars if v.name.startswith("G")]

Doptimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(Dloss, var_list=Dvars)
Goptimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(Gloss, var_list=Gvars)

run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True
sess = tf.Session(config=run_config)
sess.run(tf.global_variables_initializer())
if usevgg:
    vgg_hr_model.load_weights(sess)
    vgg_gen_model.load_weights(sess)
graph1 = sess.graph
saver_final = tf.train.Saver(max_to_keep=30)



INFO:tensorflow:Restoring parameters from /tmp/tmp115kxm99
INFO:tensorflow:Restoring parameters from /tmp/tmppl3mw3d1


In [9]:
#Train
model_prefix = './trained_model/srgan_%d.model'%downsample
t0 = time.time()
limit = 0
dlosslimit = 0
c_logits_fake = c_logits_real = dloss = gloss = 0.0
dloss = gloss = None
telegram_send.send(messages=['Model started.'])
with open('trained_images/0_hr.png', "rb") as openimg:
    telegram_send.send(images=[openimg])
with open('trained_images/0_lr.png', "rb") as openimg:
    telegram_send.send(images=[openimg])
for e in range(EPOCH):
    perm = np.random.permutation(x_train.shape[0])
    dtloss = gtloss = 0.0
    dskip = gskip = 0
    for i in range(PERIOD):
        gloss_flag = dloss_flag = True
        train_batch = x_train[perm[i*mb_size:(i+1)*mb_size]]
        imgs_hr, imgs_lr = list(zip(*map(img_crop_downsample, train_batch, np.ones(mb_size)*downsample)))
        imgs_hr = np.asarray(imgs_hr)
        imgs_lr = np.asarray(imgs_lr)

        if dloss:
            if dloss < dlosslimit and dlosslimit != 0:
                print("STOP D TRAINING")
                dloss_flag = False
                dskip += 1
            
        if dloss_flag:
            _, dloss = sess.run([Doptimizer, Dloss], feed_dict={X:imgs_hr, Z:imgs_lr})
        else:
            dloss= sess.run(Dloss, feed_dict={X:imgs_hr, Z:imgs_lr})
        if gloss_flag:
            _, mse_loss, vgg_loss, gloss, generated_img = sess.run([Goptimizer, mse, vggloss, Gloss, G], feed_dict={X:imgs_hr, Z:imgs_lr})
        else:
            vgg_loss, mse_loss, gloss, generated_img = sess.run([mse, vggloss, Gloss, G], feed_dict={X:imgs_hr, Z:imgs_lr})
        
        #Save images (interval of 2)
        if i%10 == 0 and (e == 0 or e == 1):
            save_gen_hr = sess.run(G, feed_dict={X:save_imgs_hr, Z:save_imgs_lr})
            for f in range(2):
                save_imgset(save_imgs_lr[f*4:(f+1)*4],save_gen_hr[f*4:(f+1)*4],save_imgs_hr[f*4:(f+1)*4],'./trained_images/%d_e%d_%05d'%(f,e,i),e)
        elif i%50 == 0:
            save_gen_hr = sess.run(G, feed_dict={X:save_imgs_hr, Z:save_imgs_lr})
            for f in range(2):
                save_imgset(save_imgs_lr[f*4:(f+1)*4],save_gen_hr[f*4:(f+1)*4],save_imgs_hr[f*4:(f+1)*4],'./trained_images/%d_e%d_%05d'%(f,e,i),e)

        dtloss += dloss
        gtloss += gloss
        print("Period:",i,"dloss:",dloss,"gloss:",gloss, "vggloss:", vgg_loss, "mse:", mse_loss)
        if math.isnan(dtloss) or math.isnan(gtloss):
            sess.run(tf.initialize_all_variables()) # initialize & retry if NaN
            print("...initialize parameters for nan...")
            dtloss = gtloss = 0.0
        
    endtime = int(time.time() - t0)
    print("%d: dloss=%.3f, gloss=%.3f,dskip=%d, gskip=%d, time=%dmin %ds" % (e+1, dtloss / PERIOD, gtloss / PERIOD,  dskip, gskip, endtime / 60,endtime % 60))
    if e%SAVE_INTERVAL == 0:
        saver_final.save(sess,model_prefix, global_step= e)
        print("Checkpoint saved.")
    
    if e%5 == 0:
        telegram_send.send(messages=['Epoch: %d, dloss: %f, gloss: %f'%(e,dloss,gloss)])
        with open('trained_images/%d_e%d_%05d.png'%(0,e,0), "rb") as openimg:
            telegram_send.send(images=[openimg])

saver_final.save(sess, model_prefix)
telegram_send.send(messages=['Model Finished.'])

/home/student03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if __name__ == '__main__':
/home/student03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/student03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


Period: 0 dloss: 1.36063 gloss: 0.718789 vggloss: 0.338794 mse: 0.361829
Period: 1 dloss: 2.00565e-05 gloss: 0.703978 vggloss: 0.352653 mse: 0.331609
Period: 2 dloss: 2.5923e-07 gloss: 0.754612 vggloss: 0.3874 mse: 0.343413
Period: 3 dloss: 6.84555e-09 gloss: 0.684091 vggloss: 0.34147 mse: 0.314297
Period: 4 dloss: 2.13319e-09 gloss: 0.593245 vggloss: 0.330917 mse: 0.231312
Period: 5 dloss: 4.74942e-09 gloss: 0.639308 vggloss: 0.330415 mse: 0.276365
Period: 6 dloss: 1.4759e-10 gloss: 0.725342 vggloss: 0.438545 mse: 0.24988
Period: 7 dloss: 2.08078e-12 gloss: 0.587317 vggloss: 0.330068 mse: 0.216575
Period: 8 dloss: 7.83615e-15 gloss: 0.547924 vggloss: 0.321309 mse: 0.181086
Period: 9 dloss: 9.1694e-19 gloss: 0.544591 vggloss: 0.324305 mse: 0.171116
Period: 10 dloss: 3.54831e-20 gloss: 0.576199 vggloss: 0.342041 mse: 0.182102
Period: 11 dloss: 2.61589e-19 gloss: 0.632405 vggloss: 0.356292 mse: 0.221551
Period: 12 dloss: 4.91673e-22 gloss: 0.646901 vggloss: 0.391073 mse: 0.199802
Period:

Period: 105 dloss: 3.08615e-28 gloss: 0.282667 vggloss: 0.17557 mse: 0.0332855
Period: 106 dloss: 7.17388e-28 gloss: 0.249458 vggloss: 0.150547 mse: 0.0250041
Period: 107 dloss: 3.7419e-28 gloss: 0.2923 vggloss: 0.175458 mse: 0.0431257
Period: 108 dloss: 2.54102e-26 gloss: 0.293627 vggloss: 0.186966 mse: 0.0337586
Period: 109 dloss: 1.90695e-25 gloss: 0.297646 vggloss: 0.193733 mse: 0.0313657
Period: 110 dloss: 1.73592e-26 gloss: 0.258897 vggloss: 0.159767 mse: 0.027163
Period: 111 dloss: 1.61008e-28 gloss: 0.275009 vggloss: 0.172724 mse: 0.0285234
Period: 112 dloss: 1.2957e-29 gloss: 0.284707 vggloss: 0.17495 mse: 0.0361084
Period: 113 dloss: 1.98819e-29 gloss: 0.272364 vggloss: 0.163944 mse: 0.0343296
Period: 114 dloss: 6.80872e-27 gloss: 0.321286 vggloss: 0.213846 mse: 0.0343519
Period: 115 dloss: 8.7946e-28 gloss: 0.284759 vggloss: 0.1682 mse: 0.0430184
Period: 116 dloss: 6.75036e-27 gloss: 0.286744 vggloss: 0.172967 mse: 0.0399446
Period: 117 dloss: 1.38457e-29 gloss: 0.254559 vgg

Period: 208 dloss: 4.32553e-26 gloss: 0.301204 vggloss: 0.186504 mse: 0.0429687
Period: 209 dloss: 1.26913e-28 gloss: 0.296094 vggloss: 0.178641 mse: 0.0434703
Period: 210 dloss: 4.43682e-27 gloss: 0.253353 vggloss: 0.148059 mse: 0.0332333
Period: 211 dloss: 3.40352e-26 gloss: 0.218903 vggloss: 0.118972 mse: 0.0266575
Period: 212 dloss: 5.27196e-27 gloss: 0.220014 vggloss: 0.123864 mse: 0.022837
Period: 213 dloss: 2.31686e-28 gloss: 0.270321 vggloss: 0.166232 mse: 0.0318864
Period: 214 dloss: 1.02407e-28 gloss: 0.264569 vggloss: 0.153022 mse: 0.0385308
Period: 215 dloss: 2.64211e-26 gloss: 0.232026 vggloss: 0.125054 mse: 0.034275
Period: 216 dloss: 5.0586e-28 gloss: 0.289771 vggloss: 0.169644 mse: 0.0476622
Period: 217 dloss: 2.34679e-28 gloss: 0.291074 vggloss: 0.183012 mse: 0.0356138
Period: 218 dloss: 2.23763e-29 gloss: 0.221736 vggloss: 0.123893 mse: 0.0247311
Period: 219 dloss: 1.39655e-25 gloss: 0.286454 vggloss: 0.178905 mse: 0.035407
Period: 220 dloss: 2.0213e-28 gloss: 0.24366

Period: 312 dloss: 1.87221e-25 gloss: 0.212451 vggloss: 0.117473 mse: 0.0229305
Period: 313 dloss: 1.36384e-27 gloss: 0.240321 vggloss: 0.134482 mse: 0.0335183
Period: 314 dloss: 1.20481e-28 gloss: 0.229869 vggloss: 0.134348 mse: 0.0232855
Period: 315 dloss: 3.75351e-28 gloss: 0.2332 vggloss: 0.13971 mse: 0.0219148
Period: 316 dloss: 1.39848e-26 gloss: 0.265191 vggloss: 0.163101 mse: 0.0300402
Period: 317 dloss: 2.30179e-27 gloss: 0.247099 vggloss: 0.14773 mse: 0.0277022
Period: 318 dloss: 5.71496e-27 gloss: 0.275266 vggloss: 0.170587 mse: 0.0328053
Period: 319 dloss: 1.04951e-27 gloss: 0.243502 vggloss: 0.141383 mse: 0.0301829
Period: 320 dloss: 1.34679e-26 gloss: 0.249992 vggloss: 0.149722 mse: 0.0275611
Period: 321 dloss: 1.00441e-24 gloss: 0.19915 vggloss: 0.107854 mse: 0.0185028
Period: 322 dloss: 2.38924e-27 gloss: 0.212666 vggloss: 0.120362 mse: 0.021032
Period: 323 dloss: 2.04494e-25 gloss: 0.227805 vggloss: 0.136617 mse: 0.0196488
Period: 324 dloss: 6.61331e-27 gloss: 0.232141

...

Period: 335 dloss: 3.40111e-18 gloss: 0.167903 vggloss: 0.0941097 mse: 0.0234351
Period: 336 dloss: 2.60039e-19 gloss: 0.165637 vggloss: 0.0864384 mse: 0.0261204
Period: 337 dloss: 6.13394e-19 gloss: 0.148583 vggloss: 0.0771988 mse: 0.0205642
Period: 338 dloss: 1.76712e-19 gloss: 0.165147 vggloss: 0.0895335 mse: 0.0225476
Period: 339 dloss: 1.42094e-19 gloss: 0.174062 vggloss: 0.0997096 mse: 0.0226533
Period: 340 dloss: 2.83286e-19 gloss: 0.139621 vggloss: 0.0706378 mse: 0.0174435
Period: 341 dloss: 1.14695e-18 gloss: 0.146145 vggloss: 0.0732232 mse: 0.0198222
Period: 342 dloss: 3.64e-20 gloss: 0.143448 vggloss: 0.0717618 mse: 0.0180114
Period: 343 dloss: 5.08733e-21 gloss: 0.142462 vggloss: 0.0709309 mse: 0.0186158
Period: 344 dloss: 3.80316e-19 gloss: 0.186255 vggloss: 0.110758 mse: 0.0235224
Period: 345 dloss: 8.59645e-20 gloss: 0.16266 vggloss: 0.0908704 mse: 0.0207899
Period: 346 dloss: 1.71257e-19 gloss: 0.171521 vggloss: 0.0969966 mse: 0.0230097
Period: 347 dloss: 2.19708e-19 gl

Period: 437 dloss: 5.04493e-18 gloss: 0.161916 vggloss: 0.0887263 mse: 0.0226453
Period: 438 dloss: 5.31342e-18 gloss: 0.137768 vggloss: 0.0665601 mse: 0.0203657
Period: 439 dloss: 3.43431e-19 gloss: 0.179541 vggloss: 0.103071 mse: 0.0248752
Period: 440 dloss: 1.8821e-17 gloss: 0.147991 vggloss: 0.0766075 mse: 0.0206272
Period: 441 dloss: 1.37989e-20 gloss: 0.174593 vggloss: 0.0954568 mse: 0.0247514
Period: 442 dloss: 1.10099e-17 gloss: 0.179984 vggloss: 0.103157 mse: 0.0246675
Period: 443 dloss: 1.67082e-20 gloss: 0.14477 vggloss: 0.0717886 mse: 0.0172729
Period: 444 dloss: 1.15943e-18 gloss: 0.163858 vggloss: 0.0896373 mse: 0.0225087
Period: 445 dloss: 8.61224e-20 gloss: 0.155863 vggloss: 0.081205 mse: 0.0215248
Period: 446 dloss: 1.66391e-19 gloss: 0.155067 vggloss: 0.0844288 mse: 0.0189662
Period: 447 dloss: 1.52953e-19 gloss: 0.180563 vggloss: 0.104787 mse: 0.0229665
Period: 448 dloss: 1.64939e-19 gloss: 0.159467 vggloss: 0.0849896 mse: 0.0233835
Period: 449 dloss: 1.38294e-18 glo

Period: 539 dloss: 1.37959e-20 gloss: 0.159205 vggloss: 0.0829173 mse: 0.0209169
Period: 540 dloss: 7.16865e-20 gloss: 0.167728 vggloss: 0.0913333 mse: 0.0215815
Period: 541 dloss: 3.55561e-19 gloss: 0.176518 vggloss: 0.0994856 mse: 0.0242322
Period: 542 dloss: 9.53518e-20 gloss: 0.148005 vggloss: 0.077319 mse: 0.0177515
Period: 543 dloss: 2.24894e-18 gloss: 0.198431 vggloss: 0.118115 mse: 0.0280998
Period: 544 dloss: 3.50746e-18 gloss: 0.154381 vggloss: 0.0814073 mse: 0.0213173
Period: 545 dloss: 6.30856e-19 gloss: 0.161764 vggloss: 0.0900237 mse: 0.0204145
Period: 546 dloss: 9.01563e-20 gloss: 0.184059 vggloss: 0.106127 mse: 0.0261771
16: dloss=0.000, gloss=0.166,dskip=0, gskip=0, time=172min 36s
Period: 0 dloss: 1.9193e-19 gloss: 0.186068 vggloss: 0.10544 mse: 0.0273896
Period: 1 dloss: 3.31613e-19 gloss: 0.172683 vggloss: 0.0988865 mse: 0.0226399
Period: 2 dloss: 2.72296e-18 gloss: 0.169165 vggloss: 0.0934473 mse: 0.0237751
Period: 3 dloss: 2.90044e-19 gloss: 0.157678 vggloss: 0.08

Period: 95 dloss: 1.33809e-19 gloss: 0.149548 vggloss: 0.0753005 mse: 0.0216796
Period: 96 dloss: 1.94787e-19 gloss: 0.182617 vggloss: 0.104543 mse: 0.0243395
Period: 97 dloss: 1.60989e-20 gloss: 0.154817 vggloss: 0.0807653 mse: 0.0191369


KeyboardInterrupt: 

In [ ]:
#MSE + 1e-6VGG LOSS , DOWNSAMPLE 16

In [ ]:
saver_final.save(sess,model_prefix, global_step= e)